# Project 1: Digit Classification with KNN and Naive Bayes

In this project, you'll implement your own image recognition system for classifying digits. Read through the code and the instructions carefully and add your own code where indicated. Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but <b> please prepare your own write-up (with your own code). </b>

If you're interested, check out these links related to digit recognition:

Yann Lecun's MNIST benchmarks: http://yann.lecun.com/exdb/mnist/

Stanford Streetview research and data: http://ufldl.stanford.edu/housenumbers/

In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_mldata
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

Load the data. Notice that we are splitting the data into training, development, and test. We also have a small subset of the training data called mini_train_data and mini_train_labels that you should use in all the experiments below, unless otherwise noted.

In [ ]:
# Load the digit data either from mldata.org, or once downloaded to data_home, from disk. The data is about 53MB so this cell
# should take a while the first time your run it.
mnist = fetch_mldata('MNIST original', data_home='~/datasets/mnist')
X, Y = mnist.data, mnist.target

# Rescale grayscale values to [0,1].
X = X / 255.0

# Shuffle the input: create a random permutation of the integers between 0 and the number of data points and apply this
# permutation to X and Y.
# NOTE: Each time you run this cell, you'll re-shuffle the data, resulting in a different ordering.
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

print('data shape: ', X.shape)
print('label shape:', Y.shape)

# Set some variables to hold test, dev, and training data.
test_data, test_labels = X[61000:], Y[61000:]
dev_data, dev_labels = X[60000:61000], Y[60000:61000]
train_data, train_labels = X[:60000], Y[:60000]
mini_train_data, mini_train_labels = X[:1000], Y[:1000]

(1) Create a 10x10 grid to visualize 10 examples of each digit. Python hints:

- plt.rc() for setting the colormap, for example to black and white
- plt.subplot() for creating subplots
- plt.imshow() for rendering a matrix
- np.array.reshape() for reshaping a 1D feature vector into a 2D matrix (for rendering)

In [ ]:
#def P1(num_examples=10):

### STUDENT START ###
## TODO: cosmetic changes, like changing color and size
# store indices for 10 examples of each digit in dictionary
examples_dict = {num: [] for num in range(10)}
sets_found = set()
for i, label in enumerate(mini_train_labels):
    if len(sets_found) == 10:
        break
    if len(examples_dict[label]) < 10:
        examples_dict[label].append(i)
    else:
        sets_found.add(label)

# loop over subplots and access correct matrix
# for each value in each dictionary entry
f, axarr = plt.subplots(10,10)
for i in range(10):
    for j in range(10):
        desired_matrix = mini_train_data[examples_dict[i][j]]
        im_to_show = np.reshape(desired_matrix, (28, 28))
        axarr[i,j].imshow(im_to_show)
        axarr[i,j].axis('off')
### STUDENT END ###

#P1(10)

(2) Evaluate a K-Nearest-Neighbors model with k = [1,3,5,7,9] using the mini training set. Report accuracy on the dev set. For k=1, show precision, recall, and F1 for each label. Which is the most difficult digit?

- KNeighborsClassifier() for fitting and predicting
- classification_report() for producing precision, recall, F1 results

In [ ]:
#def P2(k_values):

### STUDENT START ###
k_values = [1, 3, 5, 7, 9]
for k in k_values:
    k_neighbors = KNeighborsClassifier(k)
    k_neighbors.fit(mini_train_data, mini_train_labels)
    dev_predictions = k_neighbors.predict(dev_data)
    print("Accuracy when k={}: ".format(k), np.sum(dev_predictions==dev_labels) / 1000)
    if k==1:
        print("Classification report when k=1:")
        print(classification_report(dev_labels, dev_predictions))

    
### STUDENT END ###

#k_values = [1, 3, 5, 7, 9]
#P2(k_values)

ANSWER: The most difficult digit for precision is 3, which means that of all the digits the model predicts to be 3's, a smaller percentage of those predictions are correct compared to the other labels. The most difficult digit for recall is 2, which means that the model incorrectly predicts on more 2's than it does on other labels. Overall, the digit with the lowest f1 score (combination of precision and recall) is 9.

(3) Using k=1, report dev set accuracy for the training set sizes below. Also, measure the amount of time needed for prediction with each training size.

- time.time() gives a wall clock value you can use for timing operations

In [ ]:
#def P3(train_sizes, accuracies):

### STUDENT START ###

train_sizes = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25000]
accuracies = []
for size in train_sizes:
    k_neighbors = KNeighborsClassifier(1)
    k_neighbors.fit(train_data[:size], train_labels[:size])
    start_time = time.time()
    dev_predictions = k_neighbors.predict(dev_data)
    end_time = time.time()
    accuracy = np.sum(dev_predictions==dev_labels) / 1000
    accuracies.append(accuracy)
    prediction_time = end_time - start_time
    print("Training set size: {}".format(size))
    print("Accuracy: {}".format(accuracy))
    print("Time taken for 1000 predictions: {:.2} seconds\n".format(prediction_time))
    


### STUDENT END ###

#train_sizes = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25000]
#accuracies = []
#P3(train_sizes, accuracies)

(4) Fit a regression model that predicts accuracy from training size. What does it predict for n=60000? What's wrong with using regression here? Can you apply a transformation that makes the predictions more reasonable?

- Remember that the sklearn fit() functions take an input matrix X and output vector Y. So each input example in X is a vector, even if it contains only a single value.

In [ ]:
#def P4():

### STUDENT START ###

## TODO: talk to study group: what kind of transformation would work here??

# turn train sizes list into array and reshape into 2-d matrix
train_sizes_array = np.asarray(train_sizes)
train_sizes_array = np.reshape(train_sizes_array, (len(train_sizes_array), 1))

# create and fit linear regression model
lr = LinearRegression()
lr.fit(train_sizes_array, accuracies)

lr.predict(60000)

### STUDENT END ###

#P4()

ANSWER: When we regress accuracy on training examples, the model predicts that a training size of 60,000 will lead to an accuracy of 1.243, which is not possible, since the most highest accuracy a model can have is 1 (100% accurate). We need to apply a transformation to the output variable such that the accuracy values will be modeled as approaching 1.

Fit a 1-NN and output a confusion matrix for the dev data. Use the confusion matrix to identify the most confused pair of digits, and display a few example mistakes.

- confusion_matrix() produces a confusion matrix

In [ ]:
#def P5():

### STUDENT START ###

# train, fit, and predict on the classifier
k_neighbors = KNeighborsClassifier(1)
k_neighbors.fit(train_data[:6400], train_labels[:6400])
dev_predictions = k_neighbors.predict(dev_data)
cm = confusion_matrix(dev_labels, dev_predictions)
print(cm)

# The most commonly confused pairs of digits are: 
# 9's get confused for 7's. 

# find all occurrences where a true 9 is predicted as a 7
true_9s = np.where(dev_labels==9)
predicted_7s = np.where(dev_predictions==7)
true_9s_predicted_7s = np.intersect1d(true_9s, predicted_7s)

# display the examples
f, axarr = plt.subplots(1,len(true_9s_predicted_7s))
for i, idx in enumerate(true_9s_predicted_7s):
    desired_matrix = dev_data[idx]
    im_to_show = np.reshape(desired_matrix, (28, 28))
    axarr[i].imshow(im_to_show)
    axarr[i].axis('off')
'''
# trying to plot it, but would need to figure out
# how to put the numbers on it without itertools

plt.figure(figsize=(10, 10))
plt.title("Confusion Matrix")
plt.imshow(cm, cmap=plt.cm.Greys)'''


### STUDENT END ###

#P5()

(6) A common image processing technique is to smooth an image by blurring. The idea is that the value of a particular pixel is estimated as the weighted combination of the original value and the values around it. Typically, the blurring is Gaussian -- that is, the weight of a pixel's influence is determined by a Gaussian function over the distance to the relevant pixel.

Implement a simplified Gaussian blur by just using the 8 neighboring pixels: the smoothed value of a pixel is a weighted combination of the original value and the 8 neighboring values. Try applying your blur filter in 3 ways:
- preprocess the training data but not the dev data
- preprocess the dev data but not the training data
- preprocess both training and dev data

Note that there are Guassian blur filters available, for example in scipy.ndimage.filters. You're welcome to experiment with those, but you are likely to get the best results with the simplified version I described above.

In [ ]:
#def P6():
    
### STUDENT START ###


### STUDENT END ###

#P6()

ANSWER:

(7) Fit a Naive Bayes classifier and report accuracy on the dev data. Remember that Naive Bayes estimates P(feature|label). While sklearn can handle real-valued features, let's start by mapping the pixel values to either 0 or 1. You can do this as a preprocessing step, or with the binarize argument. With binary-valued features, you can use BernoulliNB. Next try mapping the pixel values to 0, 1, or 2, representing white, grey, or black. This mapping requires MultinomialNB. Does the multi-class version improve the results? Why or why not?

In [ ]:
#def P7():

### STUDENT START ###


    
### STUDENT END ###

#P7()

ANSWER:

(8) Use GridSearchCV to perform a search over values of alpha (the Laplace smoothing parameter) in a Bernoulli NB model. What is the best value for alpha? What is the accuracy when alpha=0? Is this what you'd expect?

- Note that GridSearchCV partitions the training data so the results will be a bit different than if you used the dev data for evaluation.

In [ ]:
#def P8(alphas):

### STUDENT START ###



### STUDENT END ###

#alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
#nb = P8(alphas)

In [ ]:
#print nb.best_params_

ANSWER:

(9) Try training a model using GuassianNB, which is intended for real-valued features, and evaluate on the dev data. You'll notice that it doesn't work so well. Try to diagnose the problem. You should be able to find a simple fix that returns the accuracy to around the same rate as BernoulliNB. Explain your solution.

Hint: examine the parameters estimated by the fit() method, theta\_ and sigma\_.

In [ ]:
#def P9():

### STUDENT END ###


### STUDENT END ###

#gnb = P9()

ANSWER:

(10) Because Naive Bayes is a generative model, we can use the trained model to generate digits. Train a BernoulliNB model and then generate a 10x20 grid with 20 examples of each digit. Because you're using a Bernoulli model, each pixel output will be either 0 or 1. How do the generated digits compare to the training digits?

- You can use np.random.rand() to generate random numbers from a uniform distribution
- The estimated probability of each pixel is stored in feature\_log\_prob\_. You'll need to use np.exp() to convert a log probability back to a probability.

In [ ]:
#def P10(num_examples):

### STUDENT START ###


### STUDENT END ###

#P10(20)

ANSWER:

(11) Remember that a strongly calibrated classifier is rougly 90% accurate when the posterior probability of the predicted class is 0.9. A weakly calibrated classifier is more accurate when the posterior is 90% than when it is 80%. A poorly calibrated classifier has no positive correlation between posterior and accuracy.

Train a BernoulliNB model with a reasonable alpha value. For each posterior bucket (think of a bin in a histogram), you want to estimate the classifier's accuracy. So for each prediction, find the bucket the maximum posterior belongs to and update the "correct" and "total" counters.

How would you characterize the calibration for the Naive Bayes model?

In [ ]:
#def P11(buckets, correct, total):
    
### STUDENT START ###


                
### STUDENT END ###

#buckets = [0.5, 0.9, 0.999, 0.99999, 0.9999999, 0.999999999, 0.99999999999, 0.9999999999999, 1.0]
#correct = [0 for i in buckets]
#total = [0 for i in buckets]

#P11(buckets, correct, total)

#for i in range(len(buckets)):
#    accuracy = 0.0
#    if (total[i] > 0): accuracy = correct[i] / total[i]
#    print 'p(pred) <= %.13f    total = %3d    accuracy = %.3f' %(buckets[i], total[i], accuracy)

ANSWER:

(12) EXTRA CREDIT

Try designing extra features to see if you can improve the performance of Naive Bayes on the dev set. Here are a few ideas to get you started:
- Try summing the pixel values in each row and each column.
- Try counting the number of enclosed regions; 8 usually has 2 enclosed regions, 9 usually has 1, and 7 usually has 0.

Make sure you comment your code well!

In [ ]:
#def P12():

### STUDENT START ###


### STUDENT END ###

#P12()